相较于前面的padding和stride，channel其实才是CNN里面大家通常会认真去设计的一个超参数

多个输入通道实际上对应的也就是，一个像素点会具有不同的颜色，比如RGB三个通道，转化为灰度图像自然只有1个通道，但是肯定丢掉信息
* 也就是说，输入的单张图片不再是一个矩阵，而是一个3d的tensor

在计算上，我们对每一个通道单独有一个卷积核，最后的输出是多个通道卷积之后的结果相加
* 实际上卷积的那个互相关公式依旧是没有变化的，但是输入和卷积核都会多一个维度，也就是通道
* 输出还是单通道，因为我们较为简单的情况下不关心多通道输出

当然，有时候我们需要具有多通道的输出，其实类比前面的，我们很容易得到答案。
* 前面为了解决多通道的输入问题，我们引入了三维卷积核，也就是每个通道的输入都对应一个二维卷积核，这样能够处理每个通道的输入
* 类比的，我们为了解决多通道输出的问题，引入四维卷积核，也就是说每一个三维卷积核对应了一个通道的输出
* 所以，具有多通道输入输出的网络，卷积核必定是4d的

多输出通道其实是在分别识别图像中的特定模式，而多输入通道则是识别并且组合图片中的特征，这里可以结合之前对卷积核的举例来理解，也就是一些卷积核有锐化作用，另一些有模糊作用

还记得吗，我们前面提到过，卷积核一般选奇数而不选偶数，最常用的是33 55这两个卷积核，但是实际上，11卷积核也很受欢迎
* 从空间结构来看，11卷积核其实并不能提取任何的空间信息，因为11卷积核只看像素点本身，等于压根没考虑空间信息
* 那为啥11卷积核会受到欢迎呢？因为11卷积核不看空间信息的特点，非常方便做通道融合
* 比如输出3通道变成2通道，可以设计一个2311大小的思维卷积核，由6个11卷积核组成，画个图就很容易发现，这个网络把原来的3通道融合成了2通道，但是空间的信息可以完全保留，不会受到影响
* 实际上这玩意会有点像，全连接层，因为完全没考虑空间信息，实际上就是全连接层

这一节重点是概念理解，搞清楚这些维度的关系，以及为啥要有通道这个事情，下面是代码的实现

多输入通道计算的实现

In [13]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    # 也就是多输入通道，做互相关运算
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [14]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

多输出通道的计算实现

In [15]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [16]:
K = torch.stack((K, K + 1, K + 2), 0) # 最后这个0，是表示堆叠的维度
K.shape

torch.Size([3, 2, 2, 2])

In [17]:
# 第一个通道还是k，是一样的
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

下面来试试看11卷积核的运算

In [18]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [20]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6
# 可以发现就是一个全连接层，corr2d_multi_in_out_1x1这里面就是dense layer的实现，和卷积没半毛钱关系

上面就是如何手写实现一个多通道输入输出的卷积层，如果用框架的话，那么直接conv2d实例化的时候指定就好了，前面的小节也有体现出来

李沐老师的答疑中有提到，对于不同通道的卷积核来说，其实可以是大小不一样的，但是通常会选择大小一样的，这是为了计算上能够更加方便的实现

要注意，我们这里实际上讨论的都是2d的卷积操作，因为我们输入的图像本质上都是2d的，虽然有通道，但是并不是图像自身的尺寸信息。如果是一些具有立体结构的图像，那就要3d卷积，多通道情况下卷积核就是5d的了